<h1>Wirecell Clustering v0_xx_0 (sbndcode v0x_xx_xx)</h1>

[General commands (Jupyter, Github, etc)](http://localhost:8888/notebooks/OneDrive/Documentos/backup-2023-xx-xx/Meetings/WireCell/Misc/Misc.ipynb)

<h5>Code locations (make sure to push changes to personal github frequently)

[gpvm] /exp/sbnd/app(data)/users/ebatista/wirecell-clustering <br>
[Ewerton local] cd /mnt/c/Users/ewert/OneDrive/Documentos/backup-2023-xx-xx/Meetings/WireCell/Clustering/wirecell-clustering <br>
[Ewerton GitHub] https://github.com/ebelchio12/wirecell-clustering <br>
 <br>
[Haiwang: try this] https://github.com/HaiwangYu/wcp-porting-img/tree/main/sbnd <br>
[Haiwang: input files] /exp/sbnd/app/users/yuhw/wcp-porting-img/sbnd/ <br>
[Haiwang: setup] https://github.com/HaiwangYu/wcp-porting-img/blob/main/sbnd/setup.sh

<h5>Setup sbndcode

In [ ]:
/cvmfs/oasis.opensciencegrid.org/mis/apptainer/current/bin/apptainer shell --shell=/bin/bash -B /cvmfs,/exp,/nashome,/pnfs/sbn,/pnfs/sbnd,/opt,/run/user,/etc/hostname,/etc/hosts,/etc/krb5.conf --ipc --pid /cvmfs/singularity.opensciencegrid.org/fermilab/fnal-dev-sl7:latest

#%%bash 
source /cvmfs/sbnd.opensciencegrid.org/products/sbnd/setup_sbnd.sh
setup sbndcode v09_91_02 -q e26:prof

***
<h5>Dec 20, 2024</h5>
0-0.6 GeV low energy range<br>
0.6-1.5 medium<br>
1.5-3.0 high<br>
 
10 to 20 events of:
0. Cosmics 
1. NumuCC
2. nueCC
3. NC

Create machinery from gen to 3D bee display to save JSON info
 
1. Handscan
2. Quantitative
3. Completeness/Efficiciency

Check how to save MC info to make 3D view in bee display 

***
<h5>Jan 7, 2025</h5>

In [ ]:
#!/bin/bash
# usage: ./run-eval.sh

# gen FCL
FCL_GEN0=prodgenie_enuelastic.fcl
FCL_GEN1=prodcorsika_cosmics_cmc.fcl
FCL_GEN2=prodgenie_intrnue_singleinteraction_tpc_sbnd.fcl
FCL_GEN3=prodgenie_nu_singleinteraction_tpc_sbnd.fcl
FCL_GEN4=NC_gen.fcl

# choose process and nevents
FCL_GEN=$FCL_GEN0
NEVT=20

# set DIR
WORKDIR=/exp/sbnd/app/users/ebatista/wirecell-clustering/work
OUTDIR=/exp/sbnd/data/users/ebatista/wirecell-clustering/out

# other FCL
FCL_G4=g4_sce_lite.fcl
FCL_SIGPROC=wirecell_sim_sp_sbnd.fcl
FCL_IMGCLUS=wcls-img-clus.fcl

if [ "$FCL_GEN" == "$FCL_GEN0" ]; then
  OUTDIR=$OUTDIR/enuelastic
elif [ "$FCL_GEN" == "$FCL_GEN1" ]; then
  OUTDIR=$OUTDIR/cosmics
elif [ "$FCL_GEN" == "$FCL_GEN2" ]; then
  OUTDIR=$OUTDIR/nueCC
elif [ "$FCL_GEN" == "$FCL_GEN3" ]; then
  OUTDIR=$OUTDIR/numuCC
elif [ "$FCL_GEN" == "$FCL_GEN4" ]; then
  OUTDIR=$OUTDIR/NC
fi

# clone img-clus repo
cd $WORKDIR
if [ ! -d "wcp-porting-img" ]; then
  git clone https://github.com/HaiwangYu/wcp-porting-img.git
fi

# setup
/cvmfs/oasis.opensciencegrid.org/mis/apptainer/current/bin/apptainer shell --shell=/bin/bash -B /cvmfs,/exp,/nashome,/pnfs/sbn,/pnfs/sbnd,/opt,/run/user,/etc/hostname,/etc/hosts,/etc/krb5.conf --ipc --pid /cvmfs/singularity.opensciencegrid.org/fermilab/fnal-dev-sl7:latest
source /cvmfs/sbnd.opensciencegrid.org/products/sbnd/setup_sbnd.sh
setup sbndcode v09_91_02 -q e26:prof

# gen
lar -n $NEVT -c $FCL_GEN -o $OUTDIR/gen_${NEVT}evts.root #>& $OUTDIR/gen.log

# g4
lar -c $FCL_G4 -s $OUTDIR/gen_${NEVT}evts.root -o $OUTDIR/g4_${NEVT}evts.root #>& $OUTDIR/g4.log

# sigproc
lar -c $FCL_SIGPROC -s $OUTDIR/g4_${NEVT}evts.root -o $OUTDIR/sigproc_${NEVT}evts.root #>& $OUTDIR/sigproc.log

# img-clus
cd $WORKDIR/wcp-porting-img/sbnd
source setup.sh
time lar --nskip 1 -n 1 -c $FCL_IMGCLUS -s $OUTDIR/sigproc_${NEVT}evts.root -o $OUTDIR/imgclus_${NEVT}evts.root #>& $OUTDIR/imgclus.log
python merge-zip.py
../upload-to-bee.sh mabc.zip #>& $OUTDIR/bee.log
mv *.zip $OUTDIR
mv *.tar.gz $OUTDIR
rm cputime.db memory.db messages.log errors.log hists_*root sbnd-data-check.root

In [ ]:
#!/bin/bash
# usage: ./run-clus.sh

# set DIR
WORKDIR=/exp/sbnd/app/users/ebatista/wirecell-clustering/work
OUTDIR=/exp/sbnd/data/users/ebatista/wirecell-clustering/out

cd $WORKDIR/wcp-porting-img/sbnd

for i in {0..19}
do
  rm clus-apa* mabc*
  lar -n 1 --nskip $i -c wcls-img-clus.fcl -s $OUTDIR/numuCC/sigproc_20evts.root -o tmp.root
  python merge-zip.py
  ../upload-to-bee.sh mabc.zip >& $OUTDIR/numuCC/bee_$i.log
done

for i in {0..19}
do
  grep list $OUTDIR/numuCC/bee_$i.log
done

***
<h5>Jan 8, 2025</h5>

TODO Jan 8, 2025

energydepos_numelectrons~500 or similar as threshold for saving truth energy deposits. Find reasonable threshold using distribution of numelectrons for all energydepos

x axis: time distributions of energy deposits -> should be something like 5 to 10 ns width

send two events to Haiwang: failed and sucessfull

------------
Later in semester!! 
cosmic tagger: tag cosmic muons (already in wirecell)
neutrino cross section measurement (inclusive)
FEMB tests with liquid nitrogen stuff in lab (regular service work)


***
<h5>Jan 9, 2025</h5>

In [ ]:
cd /exp/sbnd/app/users/ebatista/wirecell-clustering/work
mkdir larsoft_v09_91_02
cd larsoft_v09_91_02
mrb newDev
source localProducts_larsoft_v09_91_02_e26_prof/setup
cd srcs
git clone -b v09_25_04 https://github.com/LArSoft/larreco.git
mrb uc
cd $MRB_BUILDDIR
mrbsetenv
mrb i --generator ninja
cd /exp/sbnd/app/users/ebatista/wirecell-clustering/work

mrbslp

lar -n1 -c celltree_sbnd.fcl /exp/sbnd/data/users/ebatista/wirecell-clustering/out/enuelastic/sigproc_20evts.root

***
<h5>Jan 13, 2025</h5>

cd /exp/sbnd/app/users/ebatista/wirecell-clustering/work
source larsoft_v09_91_02/localProducts_larsoft_v09_91_02_e26_prof/setup

lar -n1 --nskip 2 -c celltree_sbnd.fcl /exp/sbnd/data/users/ebatista/wirecell-clustering/out/nueCC/sigproc_20evts.root
head -n 9 bee/data/0/0-truthDepo.json | tail -n 1 | cut -c 6- | tr ',' '\n' > delme.txt
a